In [ ]:
import os
import csv
import openai
from pathlib import Path
import pandas as pd

openai.api_key = "KEY_HERE"

root_dir = Path(os.getcwd())
dataset_loc = root_dir / 'database' / 'dataset.csv'

def get_prompt(statement: str, date: str) -> str:
    return  f"You're a fact-checking expert. Evaluate the truthfulness of this statement: <{statement}> written on <{date}>. Use google for research. Consider your sources, context, and date while assessing. To answer return 'Final Answer: {{verdict}}, {{reason}}. You must respond with a valid verdict ('pants-fire', 'false', 'mostly-false', 'half-true', 'mostly-true', or 'true') or 'uncertain', providing reasoning and citing sources by providing the domain of pertinent search results."
    
df = pd.read_csv(dataset_loc)
df

,Date,View,Reply,Repost,Like,Bookmark,Content,URL
0,2024.11.05,48200000,31000,98000,697000,8400,It is now officially ELECTION DAY! This will b...,https://x.com/realDonaldTrump/status/185380936...
1,2024.11.05,29300000,9600,43000,253000,4200,We are building the biggest and broadest coali...,https://x.com/realDonaldTrump/status/185357110...
2,2024.11.03,22500000,9900,35000,192000,2100,"As we rescue our economy, I will also restore ...",https://x.com/realDonaldTrump/status/185294505...
3,2024.11.03,63800000,42000,109000,691000,16000,"If Kamala wins, you are 3 days away from the s...",https://x.com/realDonaldTrump/status/185294475...
4,2024.11.01,34100000,26000,89000,388000,11000,I strongly condemn the barbaric violence again...,https://x.com/realDonaldTrump/status/185203362...
5,2024.11.01,27300000,8000,36000,208000,5600,I would like to wish our great Bitcoiners a Ha...,https://x.com/realDonaldTrump/status/185203324...
6,2024.11.01,26000000,10000,34000,238000,2200,Just watched Kamala’s Fake News Conference - A...,https://x.com/realDonaldTrump/status/185203314...
7,2024.11.01,30700000,19000,63000,272000,5500,"Lyin’ Kamala is giving a News Conference now, ...",https://x.com/realDonaldTrump/status/185203306...
8,2024.10.31,30700000,14000,42000,242000,4400,"During my Administration, we had peace in the ...",https://x.com/realDonaldTrump/status/185165852...
9,2024.10.31,10400000,2400,14000,66000,526,Derrick Anderson is running to represent the w...,https://x.com/realDonaldTrump/status/185164370...


In [ ]:
with open(file='result.csv', mode='a', encoding='utf8', newline='') as w:
    csv_writer = csv.writer(w)
    h = ['verdict', 'reason']
    dates = df['Date'].tolist()
    contents = df['Content'].tolist()

    prompt = [get_prompt(statement=content, date=date) for date, content in zip(dates, contents)]
    csv_writer.writerow(h)
    completion = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )